# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [1]:
import re
from string import punctuation
from collections import Counter, defaultdict
import textdistance
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances

In [2]:
#открываем корпус и сохраняем слова в словарь частот
corpus = open('wiki_data.txt', encoding='utf8').read()
vocab = Counter(re.findall('\w+', corpus.lower()))
#сохраняем объем корпуса
N = sum(vocab.values())

In [3]:
#сохраняем все слова из vocab в нумерованный словарь
id2word = {i:word for i, word in enumerate(vocab)}
#векторизуем слова из корпуса, признаками будут н-граммы символов от 1 до 3
vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=1000)
X = vec.fit_transform(vocab)

In [4]:
#функция с семинара для подсчета вероятностей
def P(word, N=N):
    return vocab[word] / N

#функция с семинара для нахождения близких слов (по расстоянию редактирования)
def get_closest_match_with_metric(text, lookup, topn=20, metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    return similarities.most_common(topn)

#функция с семинара для нахождения близких слов (по косинусному расстоянию)
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    return [(id2word[top], similarities[top]) for top in topn]

#функция с семинара для нахождения близких слов (сначала по косинусному расстоянию, затем по расстоянию редактирования)
def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    #мой код для нахождения наиболее вероятных слов для кандидатов с одинаковым расстоянием редактирования
    closest_reverse = defaultdict(list)
    for candidate in closest:
        closest_reverse[candidate[1]].append(candidate[0])
    closest_reverse = {key: max([i for i in value], key=P) for key, value in closest_reverse.items()}
    closest = Counter({value: key for key, value in closest_reverse.items()})

    return closest.most_common()

In [5]:
get_closest_hybrid_match('сонце', X, vec)

[('солнце', 0.8333333333333334), ('соне', 0.8)]

Посмотрим на метрики для нового алгоритма:
1) Процент правильных слов;  
2) Процент исправленных ошибок;
3) Процент ошибочно исправленных правильных слов.

In [6]:
#загружаем датасеты для тестирования алгоритма
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [7]:
#необходимые для подсчета метрик функции с семинара
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

In [8]:
#код с семинара для подсчета метрик
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec)[0][0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1
    
    if not i % 100:
        print(i)

print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0
100
200
300
400
500
600
700
800
900
0.8556278139069535
0.48835403726708076
0.09004249454461927


Процент исправленных ошибок немного увеличился по сравнению с исходным get_closest_hybrid_match (без учета вероятностей), однако наш алгоритм по метрикам все еще хуже, чем алгоритм Норвига, поэтому можно сказать, что учет вероятностей большого улучшения не дал.

Метрики для get_closest_hybrid_match с семинара:<br>
0.8527263631815908<br>
0.4658385093167702<br>
0.09004249454461927<br>

Метрики для алгоритма Норвига с семинара:<br>
0.8708354177088544<br>
0.5116459627329193<br>
0.07603077983231882<br>

## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово   
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [9]:
#1. Словарь правильных слов vocab получен еще в первом задании, он состоит из 368802 слов
print(len(vocab))

368802


In [10]:
#2. Составляем словарь удалений vocab_deletes
vocab_deletes = defaultdict(list)
for word in vocab:
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]   
    deletes = [L + R[1:] for L, R in splits if R]
    for delete in deletes:
        vocab_deletes[delete].append(word)

In [11]:
#3. Составляем функцию для исправления слова
def correct_word(word):

    #составляем для слова с опечаткой все варианты удаления
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    #добавляем в список само слово с опечаткой
    deletes.append(word)
    #выбираем среди этих вариантов те, которые встречаются в словаре удалений, сохраняем в список соответствующие им наиболее вероятные слова
    candidates = []
    for d in deletes:
        if d in vocab_deletes:
            candidates.append(max(vocab_deletes[d], key=P))
    #из правильных слов выбираем то, у которого наибольшая вероятность
    if candidates:
        return max(candidates, key=P)
    #если в словаре удалений нет подходящих вариантов исправлений, возвращаем само слово с опечаткой
    else:
        return word

Теперь посмотрим на метрики.

In [12]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:

        predicted = cashed.get(pair[1], correct_word(pair[1]))
        cashed[pair[1]] = predicted
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.43141570785392697
0.32142857142857145
0.5523142299299414


Метрики значительно хуже, чем у алгоритма Норвига. Это может быть связано с тем, что у реализованного Symspell расстояние удаления - всего 1 символ (тем не менее, при удалении 1 и 2 символов будет появляться слишком много кандидатов, среди которых почти наверняка будут высоко вероятные, это нужно будет учитывать).

## *3. Настройка гиперпараметров. (2 балла)

У метода из первого заданий много гиперпараметров (те которые нужно подбирать самостоятельно). Это параметры векторайзера, topn, метрика редактирования. Поэкспериментируйте с ними. 

Проведите как минимум 10 экспериментов с разными параметрами. Для каждого эксперимента укажите мотивацию (например, "слишком маленький topn в get_closest_match_vec приводит к тому, что некоторые хорошие варианты не доходят до get_closest_match_with_metric, попробуем его увеличить")

Старайтесь получить улучшение, но если улучшить не получится, то это не страшно. Главное, чтобы эксперименты были осмысленными, а не рандомными. 

Для начала обернем код для подсчета метрик в функцию, чтобы было удобнее их считать при проведении экспериментов.

In [13]:
#функции для подсчета метрик
def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

def metrics(true=true, bad=bad):
    
    mistakes = []
    total_mistaken = 0
    mistaken_fixed = 0

    total_correct = 0
    correct_broken = 0

    total = 0
    correct = 0

    cashed = {}
    for i in range(len(true)):
        word_pairs = align_words(true[i], bad[i])
        for pair in word_pairs:
            if predict_mistaken(pair[1], vocab):
                pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec)[0][0])
                cashed[pair[1]] = pred
            else:
                pred = pair[1]
                
            if pred == pair[0]:
                correct += 1
            else:
                mistakes.append((pair[0], pair[1], pred))
            total += 1
                
            if pair[0] == pair[1]:
                total_correct += 1
                if pair[0] != pred:
                    correct_broken += 1
            else:
                total_mistaken += 1
                if pair[0] == pred:
                    mistaken_fixed += 1

        if not i % 100:
            print(i)

    print(correct/total)
    print(mistaken_fixed/total_mistaken)
    print(correct_broken/total_correct)

1) Попробуем поэкспериментировать с метрикой редактирования. Для начала попробуем расстояние Левенштейна (в первом задании мы считали расстояние Дамерау-Левенштейна, которое учитывает не только операции вставки, удаления и замены символов, но и перестановки, однако хочется проверить, наколько сильно будут отличаться результаты, если перестановка учитываться не будет).

In [14]:
def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    closest_reverse = defaultdict(list)
    for candidate in closest:
        closest_reverse[candidate[1]].append(candidate[0])
    closest_reverse = {key: max([i for i in value], key=P) for key, value in closest_reverse.items()}
    closest = Counter({value: key for key, value in closest_reverse.items()})
    return closest.most_common()

In [15]:
metrics()

0
100
200
300
400
500
600
700
800
900
0.854327163581791
0.4782608695652174
0.09004249454461927


2) Метрики все-таки чуть-чуть похуже, чем у алгоритма с расстоянием Дамерау-Левенштейна (что, в общем-то, только подтверждает гипотезу, что необходимо учитывать операцию перестановки). Можно попробовать улучшить алгоритм за счет применения другого способа подсчета расстояния редактирования, например, с помощью меры сходства Джаро-Винклера (алгоритм работает чуть быстрее, чем расстояние Левенштейна и Дамерау-Левенштейна, но в нашем случае, скорее всего, значительного ускорения не даст, т.к. мы и уже ускорили алгоритм за счет get_closest_match_vec).

In [16]:
def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.jaro_winkler):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    closest_reverse = defaultdict(list)
    for candidate in closest:
        closest_reverse[candidate[1]].append(candidate[0])
    closest_reverse = {key: max([i for i in value], key=P) for key, value in closest_reverse.items()}
    closest = Counter({value: key for key, value in closest_reverse.items()})
    return closest.most_common()

In [17]:
metrics()

0
100
200
300
400
500
600
700
800
900
0.8502251125562781
0.44642857142857145
0.09004249454461927


Метрики оказались похуже, значит, в качестве базовой метрики редактирования оставляем расстояние Дамерау-Левенштейна.

3) В первом задании параметр topn для расчета расстояния редактирования был равен 3, а для косинусного расстояния 3*4=12. Попробуем увеличить topn, исходя из предположения, что некоторые хорошие кандидаты на этапе нахождения близких слов по косинусному расстоянию удаляются и не доходят до get_closest_match_with_metric.

In [18]:
def get_closest_hybrid_match(text, X, vec, topn=10, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    closest_reverse = defaultdict(list)
    for candidate in closest:
        closest_reverse[candidate[1]].append(candidate[0])
    closest_reverse = {key: max([i for i in value], key=P) for key, value in closest_reverse.items()}
    closest = Counter({value: key for key, value in closest_reverse.items()})
    return closest.most_common()

In [19]:
metrics()

0
100
200
300
400
500
600
700
800
900
0.8574287143571786
0.5023291925465838
0.09004249454461927


4) Метрики улучшились. Попробуем увеличить topn для косинусного расстояния. Увеличивать topn для расстояния редактирования, кажется, смысла нет, так как вряд ли среди кандидатов будет очень много тех, у которых одинаковое расстояние редактирования (по алгоритму мы выбираем кандидата с наибольшей метрикой расстояния редактирования, и вероятностный подход применяется только если таких кандидатов несколько).

In [20]:
def get_closest_hybrid_match(text, X, vec, topn=10, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*6)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    closest_reverse = defaultdict(list)
    for candidate in closest:
        closest_reverse[candidate[1]].append(candidate[0])
    closest_reverse = {key: max([i for i in value], key=P) for key, value in closest_reverse.items()}
    closest = Counter({value: key for key, value in closest_reverse.items()})
    return closest.most_common()

In [21]:
metrics()

0
100
200
300
400
500
600
700
800
900
0.8584292146073037
0.5100931677018633
0.09004249454461927


5) Так как метрики улучшились, попробуем все-таки увеличить topn еще в 2 раза (пусть и для расстояния редактирования, и для косинусного расстояния количество кандидатов увеличится).

In [22]:
def get_closest_hybrid_match(text, X, vec, topn=20, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*6)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    closest_reverse = defaultdict(list)
    for candidate in closest:
        closest_reverse[candidate[1]].append(candidate[0])
    closest_reverse = {key: max([i for i in value], key=P) for key, value in closest_reverse.items()}
    closest = Counter({value: key for key, value in closest_reverse.items()})
    return closest.most_common()

In [23]:
metrics()

0
100
200
300
400
500
600
700
800
900
0.8581290645322661
0.5077639751552795
0.09004249454461927


6) Кажется, улучшения предыдущий эксперимент не дал (значит, увеличение количества кандидатов на этапе подсчета расстояния редактирования действительно не имеет значения). Тогда попробуем оставить topn=10, а topn для косинусного расстояния еще увеличим (до 80).

In [24]:
def get_closest_hybrid_match(text, X, vec, topn=10, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*8)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    closest_reverse = defaultdict(list)
    for candidate in closest:
        closest_reverse[candidate[1]].append(candidate[0])
    closest_reverse = {key: max([i for i in value], key=P) for key, value in closest_reverse.items()}
    closest = Counter({value: key for key, value in closest_reverse.items()})
    return closest.most_common()

In [25]:
metrics()

0
100
200
300
400
500
600
700
800
900
0.8583291645822911
0.5093167701863354
0.09004249454461927


Улучшения по метрикам мы не видим, значит, дальнейшее увеличение кандидатов на этапе подсчета косинусного расстояния смысла не имеет. Остановимся на параметрах topn=10 для расстояния редактирования и topn*6 для косинусного расстояния.

7) Попробуем теперь посмотреть на параметры векторайзера. Возможно, имеет смысл увеличить max_features, так как сейчас словарь н-грамм состоит только из 1000 признаков (только для 33 букв русского алфавита (а символов в словаре наверняка больше) общий объем сочетаний 1, 2 и 3 символов составляет 33 + 33^2 + 33^3 = 37059, в нашем словаре правильных слов должно быть почти столько же). 

In [29]:
vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=5000)
X = vec.fit_transform(vocab)

In [30]:
def get_closest_hybrid_match(text, X, vec, topn=10, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*6)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    closest_reverse = defaultdict(list)
    for candidate in closest:
        closest_reverse[candidate[1]].append(candidate[0])
    closest_reverse = {key: max([i for i in value], key=P) for key, value in closest_reverse.items()}
    closest = Counter({value: key for key, value in closest_reverse.items()})
    return closest.most_common()

In [31]:
metrics()

0
100
200
300
400
500
600
700
800
900
0.8577288644322161
0.5046583850931677
0.09004249454461927


8) Метрики не улучшились. Попробуем увеличить max_features еще в 2 раза.

In [32]:
vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=10000)
X = vec.fit_transform(vocab)

In [33]:
metrics()

0
100
200
300
400
500
600
700
800
900
0.8577288644322161
0.5046583850931677
0.09004249454461927


9) Метрики остались такие же, значит, взяли слишком большое увеличение параметра, которое никак не отразилось на работе алгоритма. Попробуем остановиться на 2000 признаков.

In [34]:
vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=2000)
X = vec.fit_transform(vocab)

In [35]:
metrics()

0
100
200
300
400
500
600
700
800
900
0.8577288644322161
0.5046583850931677
0.09004249454461927


10) Улучшений по метрикам нет, поэтому теперь попробуем, наоборот, уменьшить количество признаков, например, до 800.

In [36]:
vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=800)
X = vec.fit_transform(vocab)

In [37]:
metrics()

0
100
200
300
400
500
600
700
800
900
0.8584292146073037
0.5100931677018633
0.09004249454461927


Теперь результаты не отличаются от тех, которые давал алгоритм с векторайзером, где max_feautures=1000.

По результатам экспериментов оптимальными параметрами для алгоритма будут: metric=textdistance.damerau_levenshtein, topn=10 (для get_closest_match_with_metric), topn*6 (для get_closest_match_vec) и max_features=1000 (для векторайзера).